In [1]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
from sqlalchemy import inspect
import sys
import sqlalchemy.ext.declarative as sqld
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
sys.path.append('..')

In [3]:
from modules.transforms import *

In [4]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [5]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [6]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [7]:
pop=pd.read_sql_table('producto1_Covid_19', schema='producto1',con=cnx) 
pop=pop.groupby('Region')['Poblacion'].sum()
popTOT=pop.sum()

In [8]:
popTOT

19458310.0

In [9]:
#----------------------------DEIS--------------------------------------------------------------------
s="DEIS"
n="DEIS_2020"
DEIS=pd.read_sql_table(n, con=cnx,schema=s)
DEIS=DEIS[DEIS['COVID'].notnull()]
#fallecidos nuevos regionales (DEIS)
fr=DEIS.groupby(['FECHA_DEF','Metropolitana'])['FECHA_STR'].count()
fr=fr.reset_index()
fr.columns=['Fecha','Region Metropolitana','Fallecidos Nuevos']
fr=fr.pivot(index='Fecha',columns=['Region Metropolitana'])
fr=fr.T.reset_index().T
fr.columns=fr.iloc[0].astype(str)+' '+fr.iloc[1].astype(str)+' (DEIS)'
fr=fr.iloc[3:].reset_index()
#fallecidos regional acumulado (deis)
fr=fr.fillna(0)
fra=fr[fr.columns.to_list()[1:]].cumsum()
fra.columns=[x.replace('Nuevos','Acumulados') for x in fra.columns]
#concatenar
fr=pd.concat([fr,fra],axis=1)

deis=DEIS.groupby('FECHA_DEF')['index'].count().reset_index()
deis.columns=['Fecha','Fallecidos DEIS Diarios']
deis=deis.merge(DEIS[DEIS['COVID']=='Covid PCR+'].groupby('FECHA_DEF')['index'].count().reset_index().rename(columns={
    'FECHA_DEF':'Fecha','index':'Fallecidos DEIS Diarios (PCR+)'}),on='Fecha')
deis=deis.merge(DEIS[DEIS['COVID']=='Covid Sospechoso'].groupby('FECHA_DEF')['index'].count().reset_index().rename(columns={
    'FECHA_DEF':'Fecha','index':'Fallecidos DEIS Diarios (Covid Sospechoso)'}),on='Fecha')
deis['Fallecidos DEIS (PCR+) Acumulado']=deis['Fallecidos DEIS Diarios (PCR+)'].cumsum()
deis['Fallecidos DEIS (Covid Sospechoso) Acumulado']=deis['Fallecidos DEIS Diarios (Covid Sospechoso)'].cumsum()
deis['Fallecidos DEIS Acumulado']=deis['Fallecidos DEIS Diarios'].cumsum()
deis['Fallecidos DEIS (PCR+) x Millón/Hab.']=deis['Fallecidos DEIS (PCR+) Acumulado']/(popTOT/1000000)
deis['Fallecidos DEIS (Covid Sospechoso) x Millón/Hab.']=deis['Fallecidos DEIS (Covid Sospechoso) Acumulado']/(popTOT/1000000)
deis['Fallecidos DEIS x Millón/Hab.']=deis['Fallecidos DEIS Acumulado']/(popTOT/1000000)


DEIS=deis.merge(fr,on='Fecha',how='left')
#----------------------------DEIS--------------------------------------------------------------------

In [11]:
#----------------------------ICOVID--------------------------------------------------------------------

#casos nuevos según fecha de inicio de síntomas (icovid)
s="producto53"
n="producto53_confirmados_"
nuevos=pd.read_sql_table(n, con=cnx,schema=s)
nuevos=pd.DataFrame(nuevos.groupby('fecha')['Promedio'].sum())
nuevos.columns=['Casos Nuevos (ICOVID)']
nuevos['Incidencia Nuevos (ICOVID) x 100 K Hab.']=nuevos['Casos Nuevos (ICOVID)']/popTOT*100000
nuevos['Casos Acumulados (ICOVID)']=nuevos['Casos Nuevos (ICOVID)'].cumsum()

#casos activos según fecha de inicio de síntomas (icovid)
s="producto53"
n="producto53_confirmados_regionale_casos_activos"
activos=pd.read_sql_table(n, con=cnx,schema=s)
activos=pd.DataFrame(activos.groupby('fecha')['Promedio'].sum())
activos.columns=['Casos Activos (ICOVID)']
activos['Incidencia Activos (ICOVID) x 100 K Hab.']=activos['Casos Activos (ICOVID)']/popTOT*100000

#activos regional icovid
s="producto53"
n="producto53_confirmados_regionale_casos_activos"
ar=pd.read_sql_table(n, con=cnx,schema=s)
ar=pd.DataFrame(ar.groupby(['fecha','Region_Metropolitana'])[['Poblacion','Promedio']].sum()).reset_index()
ar=ar.pivot(index='fecha',columns=['Region_Metropolitana']).T.reset_index().T
ar.columns=ar.iloc[0].astype(str)+' '+ar.iloc[1].astype(str)+' Casos Activos (ICOVID)'
ar=ar.iloc[3:].reset_index()
ar=ar.rename(columns={'fecha':'Fecha'})

#nuevos regionales icovid
s="producto53"
n="producto53_confirmados_regionale"
nr=pd.read_sql_table(n, con=cnx,schema=s)
nr=pd.DataFrame(nr.groupby(['fecha','Region_Metropolitana'])[['Poblacion','Promedio']].sum()).reset_index()
nr=nr.pivot(index='fecha',columns=['Region_Metropolitana']).T.reset_index().T
nr.columns=nr.iloc[0].astype(str)+' '+nr.iloc[1].astype(str)+' Casos Nuevos (ICOVID)'
nr=nr.iloc[3:].reset_index()
nr=nr.rename(columns={'fecha':'Fecha'})

#icovid consolidado
icovid=pd.concat([nuevos,activos],axis=1)
icovid=icovid.reset_index()
icovid=icovid.rename(columns={'fecha':'Fecha'})

icovid=icovid.merge(ar,how='left',on='Fecha').merge(nr,how='left',on='Fecha')
#----------------------------ICOVID--------------------------------------------------------------------

In [12]:
#----------------------------MINSAL--------------------------------------------------------------------

#casos MINSAL
s="producto5"
n="producto5_TotalesNacionales_T"
minsal=pd.read_sql_table(n, con=cnx,schema=s)
minsal=minsal.drop('index',axis=1)
minsal.columns=[x+' (MINSAL)' for x in minsal.columns]
minsal=minsal.rename(columns={'Fecha (MINSAL)': 'Fecha'})
minsal['Fecha']=pd.to_datetime(minsal['Fecha'],errors='coerce')

#casos MINSAL regionales
s="producto3"
n="producto3_TotalesPorRegion_std"
minsalr=pd.read_sql_table(n, con=cnx,schema=s)
#minsal=minsal.drop('index',axis=1)
#minsal.columns=[x+' (MINSAL)' for x in minsal.columns]
#minsal=minsal.rename(columns={'Fecha (MINSAL)': 'Fecha'})
#minsal['Fecha']=pd.to_datetime(minsal['Fecha'],errors='coerce')

#casos nuevos regional minsal
s="producto13"
n="producto13_CasosNuevosCumulativo_std"
nrm=pd.read_sql_table(n, con=cnx,schema=s)
nrm=nrm[nrm['Region']!='Total']
nrm=pd.DataFrame(nrm.groupby(['Fecha','Region_Metropolitana'])['Total'].sum()).reset_index()
nrm=nrm.pivot(index='Fecha',columns=['Region_Metropolitana']).T.reset_index().T
nrm.columns='Casos Nuevos (MINSAL) '+nrm.iloc[1].astype(str)
nrm=nrm.iloc[3:].reset_index()

#casos activos por fecha inicio de sintomas regional (minsal)
s="producto19"
n="producto19_CasosActivosPorComuna_std"
arms=pd.read_sql_table(n, con=cnx,schema=s)
arms=arms.fillna(0)
arms=pd.DataFrame(arms.groupby(['Fecha','Region_Metropolitana'])['Casos activos'].sum()).reset_index()
arms=arms.pivot(index='Fecha',columns=['Region_Metropolitana']).T.reset_index().T
arms.columns='Casos Activos (MINSAL) '+arms.iloc[1].astype(str)
arms=arms.iloc[3:].reset_index()

#activos regional minsal
s="producto25"
n="producto25_CasosActualesPorComuna_std"
arm=pd.read_sql_table(n, con=cnx,schema=s)
arm=arm[arm['Region']!='Total']
arm=pd.DataFrame(arm.groupby(['Fecha','Region_Metropolitana'])[['Casos actuales']].sum()).reset_index()
arm=arm.pivot(index='Fecha',columns=['Region_Metropolitana']).T.reset_index().T
arm.columns=arm.iloc[0].astype(str)+' '+arm.iloc[1].astype(str)+' (MINSAL)'
arm=arm.iloc[3:].reset_index()
#arm=arm.rename(columns={'fecha':'Fecha'})

minsal=minsal.merge(arm,how='left',on='Fecha').merge(nrm,how='left',on='Fecha').merge(arms,how='left',on='Fecha')
#----------------------------MINSAL--------------------------------------------------------------------

In [13]:
s='@perez'
n='nacional@perez'
nacional=pd.read_sql_table(n, con=cnx,schema=s)

In [14]:
nacional.sort_values(by='Fecha').dropna(subset=['Fecha'])

,index,Fecha,Test_PCR_notificados_acumulados,Laboratorios_que_reportan_en_esa_fecha,Confirmados_acumulados_totales__incluye_asintomáticos_,Confirmados_asintomáticos_acumulados_totales,Muertes_acumuladas_totales,Pacientes_actualmente_en_UCI,Pacientes_covid_positivo_actualmenteen_cama_básica,Pacientescovid_positivoactualmenteen_cama_media,...,Pacientes_sospechosos_acumulados_en_Epivigila,Confirmados_acumulados_ingresados_en_Epivigila,Pacientes_acumulados_cn_info_de_ingreso_a_hospital,Pacientes_acumulados_cn_info_de_ingreso_a_UTI/UCI,Confirmados_dossemanas_atrasmenos_muertoshasta_día_actual,Nuevos_confirmados_dos_semanas_atrás_menos_nuevos_muertos_hoy,Camas UCI Desocupadas,VMI Desocupadas,VMI no-COVID,UCI no-COVID
0,0,2020-03-06,None,nan,7.000,nan,0.000,nan,nan,nan,...,nan,nan,nan,None,None,None,nan,nan,nan,nan
1,1,2020-03-07,None,nan,9.000,nan,0.000,nan,nan,nan,...,nan,nan,nan,None,None,None,nan,nan,nan,nan
2,2,2020-03-08,None,nan,12.000,nan,0.000,nan,nan,nan,...,nan,nan,nan,None,None,None,nan,nan,nan,nan
3,3,2020-03-09,None,nan,17.000,nan,0.000,nan,nan,nan,...,nan,nan,nan,None,None,None,nan,nan,nan,nan
4,4,2020-03-10,None,nan,23.000,nan,0.000,nan,nan,nan,...,270.000,18.000,nan,None,None,None,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,235,2020-10-27,None,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,None,None,None,nan,nan,nan,nan
236,236,2020-10-28,None,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,None,None,None,nan,nan,nan,nan
237,237,2020-10-29,None,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,None,None,None,nan,nan,nan,nan
238,238,2020-10-30,None,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,None,None,None,nan,nan,nan,nan


In [16]:
nacional.shape

(240, 39)

In [17]:
minsal.shape

(274, 24)

In [18]:
DEIS.shape

(254, 14)

In [43]:
master=DEIS.merge(icovid,on='Fecha',how='outer').merge(minsal,on='Fecha',how='outer')

In [44]:
master=master.sort_values(by='Fecha')

In [45]:
master.columns=[x.replace('x','').replace('.','').replace('(','').replace(')','') for x in master.columns]

In [46]:
master

,Fecha,Fallecidos DEIS Diarios,Fallecidos DEIS Diarios PCR+,Fallecidos DEIS Diarios Covid Sospechoso,Fallecidos DEIS PCR+ Acumulado,Fallecidos DEIS Covid Sospechoso Acumulado,Fallecidos DEIS Acumulado,Fallecidos DEIS PCR+ Millón/Hab,Fallecidos DEIS Covid Sospechoso Millón/Hab,Fallecidos DEIS Millón/Hab,...,Casos probables acumulados MINSAL,Casos activos probables MINSAL,Casos nuevos sin notificar MINSAL,Casos \nactivos confirmados MINSAL,Casos actuales Región Metropolitana MINSAL,Casos actuales Resto de Chile MINSAL,Casos Nuevos MINSAL Región Metropolitana,Casos Nuevos MINSAL Resto de Chile,Casos Activos MINSAL Región Metropolitana,Casos Activos MINSAL Resto de Chile
254,2020-03-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
255,2020-03-02,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
256,2020-03-03,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
257,2020-03-04,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,NaN,NaN,1.000,1.000,NaN,NaN
258,2020-03-05,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,NaN,NaN,1.000,0.000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2020-11-28,23.000,18.000,5.000,15479.000,5126.000,20605.000,795.496,263.435,1058.931,...,75190.000,2010.000,10.000,nan,NaN,NaN,420.000,1298.000,NaN,NaN
250,2020-11-29,59.000,49.000,10.000,15528.000,5136.000,20664.000,798.014,263.949,1061.963,...,75531.000,2109.000,11.000,nan,NaN,NaN,344.000,1145.000,NaN,NaN
251,2020-11-30,42.000,30.000,12.000,15558.000,5148.000,20706.000,799.556,264.566,1064.121,...,75606.000,1933.000,19.000,nan,NaN,NaN,302.000,1011.000,NaN,NaN
252,2020-12-01,46.000,28.000,18.000,15586.000,5166.000,20752.000,800.995,265.491,1066.485,...,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
name='consolidado_nacional'
schema='nacional'
if not cnx.dialect.has_schema(cnx, schema):
    print('schema '+schema+' does not exist, creating it')
    cnx.execute(sqlalchemy.schema.CreateSchema(schema))
else:
    print('schema '+schema+' exists, will not be created')
master.to_sql(name, schema=schema,con=cnx,if_exists='replace')

schema nacional exists, will not be created


In [48]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_covid-chile.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/consolidado_nacional.ipynb')

0